In [2]:
# timing_model.py
# Defines the basic timing model interface classes
import functools
from pint.models.parameter import strParameter
from pint.phase import Phase
from astropy import log
import astropy.time as time
import numpy as np
import pint.utils as utils
import astropy.units as u
import copy
import abc
from six import add_metaclass
import astropy.coordinates as coords
import astropy.constants as const
from astropy.coordinates.angles import Angle
from astropy import log
from pint.models import parameter as p
from pint.models.timing_model import MissingParameter
from pint.utils import time_from_mjd_string, time_to_longdouble, str2longdouble
from pint.pulsar_ecliptic import PulsarEcliptic, OBL
from pint import ls
from pint import utils
import time
import matplotlib.pyplot as plt
%matplotlib inline

mas_yr = (u.mas / u.yr)

try:
    from astropy.erfa import DAYSEC as SECS_PER_DAY
except ImportError:
    from astropy._erfa import DAYSEC as SECS_PER_DAY


Define the TimingModel class

Define the component class